### 필요 라이브러리 호출

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

import yfinance as yf

from scipy.stats import ttest_ind, mannwhitneyu
import statsmodels.api as sm

import seaborn as sns
import matplotlib.pyplot as plt
import platform
if platform.system() == 'Darwin': # MAC
    plt.rc('font', family='AppleGothic')
else: # Windows
    plt.rc('font', family='Malgun Gothic')

plt.rcParams['axes.unicode_minus'] = False

In [ ]:
import utils
import preprocessing

### 데이터 수집

In [9]:
name = 'KOSPI'
# symbol = '^KS11'
symbol = '091160.KS'

# ohlcv 데이터 수집 (전체 기간)
data = preprocessing.get_data(symbol)
data

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume,rtn,event,귀속년도,rtn_1,cagr
Date,,,,,,,,,,,
2007-02-01,9045.0,9045.0,8930.0,9045.0,8742.020508,1234,1.061471,else,2007,0.010615,4.617901e+03
2007-02-02,9050.0,9340.0,9050.0,9300.0,8988.476562,5195,2.819212,else,2007,0.028192,2.553238e+06
2007-02-05,9385.0,9450.0,9320.0,9435.0,9118.954102,8512,1.451609,else,2007,0.014516,1.915364e+04
2007-02-06,9565.0,9565.0,9390.0,9440.0,9123.788086,1848,0.053010,else,2007,0.000530,2.134121e+01
2007-02-07,9440.0,9530.0,9440.0,9450.0,9133.453125,5039,0.105932,else,2007,0.001059,4.717443e+01
...,...,...,...,...,...,...,...,...,...,...,...
2023-12-28,36150.0,36310.0,35885.0,36275.0,36275.000000,367792,0.165677,santa,2023,0.001657,8.298270e+01
2024-01-02,36195.0,36930.0,35990.0,36840.0,36840.000000,902334,1.557547,santa,2023,0.015575,2.808079e+04
2024-01-03,36200.0,36400.0,35770.0,35915.0,35915.000000,760192,-2.510858,santa,2023,-0.025109,-9.999069e+01


In [10]:
# 일일수익률 계산
data = utils.create_rtn(data, col_rtn='rtn')
data

,Open,High,Low,Close,Adj Close,Volume,rtn,event,귀속년도,rtn_1,cagr
Date,,,,,,,,,,,
2007-02-02,9050.0,9340.0,9050.0,9300.0,8988.476562,5195,2.819212,else,2007,0.028192,2.553238e+06
2007-02-05,9385.0,9450.0,9320.0,9435.0,9118.954102,8512,1.451609,else,2007,0.014516,1.915364e+04
2007-02-06,9565.0,9565.0,9390.0,9440.0,9123.788086,1848,0.053010,else,2007,0.000530,2.134121e+01
2007-02-07,9440.0,9530.0,9440.0,9450.0,9133.453125,5039,0.105932,else,2007,0.001059,4.717443e+01
2009-04-17,11850.0,11850.0,10610.0,10815.0,10452.729492,10129,14.444442,else,2009,0.144444,2.438193e+23
...,...,...,...,...,...,...,...,...,...,...,...
2023-12-28,36150.0,36310.0,35885.0,36275.0,36275.000000,367792,0.165677,santa,2023,0.001657,8.298270e+01
2024-01-02,36195.0,36930.0,35990.0,36840.0,36840.000000,902334,1.557547,santa,2023,0.015575,2.808079e+04
2024-01-03,36200.0,36400.0,35770.0,35915.0,35915.000000,760192,-2.510858,santa,2023,-0.025109,-9.999069e+01


#### 데이터 기간 설정

In [11]:
date_start = '2000-01-01'
date_end = '2021-12-31'

data21 = utils.filter_date(data, date_start, date_end)
data21

,Open,High,Low,Close,Adj Close,Volume,rtn,event,귀속년도,rtn_1,cagr
Date,,,,,,,,,,,
2007-02-02,9050.0,9340.0,9050.0,9300.0,8988.476562,5195,2.819212,else,2007,0.028192,2.553238e+06
2007-02-05,9385.0,9450.0,9320.0,9435.0,9118.954102,8512,1.451609,else,2007,0.014516,1.915364e+04
2007-02-06,9565.0,9565.0,9390.0,9440.0,9123.788086,1848,0.053010,else,2007,0.000530,2.134121e+01
2007-02-07,9440.0,9530.0,9440.0,9450.0,9133.453125,5039,0.105932,else,2007,0.001059,4.717443e+01
2009-04-17,11850.0,11850.0,10610.0,10815.0,10452.729492,10129,14.444442,else,2009,0.144444,2.438193e+23
...,...,...,...,...,...,...,...,...,...,...,...
2021-12-24,39105.0,39175.0,38640.0,38700.0,38403.878906,140717,-0.154804,santa,2021,-0.001548,-4.319086e+01
2021-12-27,38725.0,38820.0,38480.0,38575.0,38279.835938,176784,-0.322996,santa,2021,-0.003230,-6.929803e+01
2021-12-28,38780.0,39120.0,38630.0,39120.0,38820.667969,207536,1.412838,santa,2021,0.014128,1.664641e+04


In [12]:
# date_start = '2000-01-01'
# date_end = '2021-12-31'

data21 = utils.filter_range(data, '귀속년도', 2000, 2021)
data21

,Open,High,Low,Close,Adj Close,Volume,rtn,event,귀속년도,rtn_1,cagr
Date,,,,,,,,,,,
2007-02-02,9050.0,9340.0,9050.0,9300.0,8988.476562,5195,2.819212,else,2007,0.028192,2.553238e+06
2007-02-05,9385.0,9450.0,9320.0,9435.0,9118.954102,8512,1.451609,else,2007,0.014516,1.915364e+04
2007-02-06,9565.0,9565.0,9390.0,9440.0,9123.788086,1848,0.053010,else,2007,0.000530,2.134121e+01
2007-02-07,9440.0,9530.0,9440.0,9450.0,9133.453125,5039,0.105932,else,2007,0.001059,4.717443e+01
2009-04-17,11850.0,11850.0,10610.0,10815.0,10452.729492,10129,14.444442,else,2009,0.144444,2.438193e+23
...,...,...,...,...,...,...,...,...,...,...,...
2022-01-10,38315.0,38340.0,37505.0,37670.0,37381.761719,121011,-2.155842,after,2021,-0.021558,-9.996491e+01
2022-01-11,37930.0,38140.0,37575.0,37790.0,37500.843750,103245,0.318556,after,2021,0.003186,2.192752e+02
2022-01-12,38305.0,38585.0,38140.0,38585.0,38289.757812,164531,2.103723,after,2021,0.021037,1.995075e+05


#### 최종 데이터 정보(요약통계량)

In [13]:
data21.head()

,Open,High,Low,Close,Adj Close,Volume,rtn,event,귀속년도,rtn_1,cagr
Date,,,,,,,,,,,
2007-02-02,9050.0,9340.0,9050.0,9300.0,8988.476562,5195,2.819212,else,2007,0.028192,2.553238e+06
2007-02-05,9385.0,9450.0,9320.0,9435.0,9118.954102,8512,1.451609,else,2007,0.014516,1.915364e+04
2007-02-06,9565.0,9565.0,9390.0,9440.0,9123.788086,1848,0.053010,else,2007,0.000530,2.134121e+01
2007-02-07,9440.0,9530.0,9440.0,9450.0,9133.453125,5039,0.105932,else,2007,0.001059,4.717443e+01
2009-04-17,11850.0,11850.0,10610.0,10815.0,10452.729492,10129,14.444442,else,2009,0.144444,2.438193e+23


In [14]:
data21.tail()

,Open,High,Low,Close,Adj Close,Volume,rtn,event,귀속년도,rtn_1,cagr
Date,,,,,,,,,,,
2022-01-10,38315.0,38340.0,37505.0,37670.0,37381.761719,121011,-2.155842,after,2021,-0.021558,-99.964906
2022-01-11,37930.0,38140.0,37575.0,37790.0,37500.843750,103245,0.318556,after,2021,0.003186,219.275176
2022-01-12,38305.0,38585.0,38140.0,38585.0,38289.757812,164531,2.103723,after,2021,0.021037,199507.453088
2022-01-13,38690.0,38900.0,38480.0,38650.0,38354.261719,133794,0.168463,after,2021,0.001685,84.849353
2022-01-14,38165.0,38890.0,38165.0,38845.0,38547.769531,175691,0.504528,after,2021,0.005045,527.702147


In [15]:
data21[['rtn']].describe()

,rtn
count,3141.000000
mean,0.060137
std,1.606437
min,-9.116665
25%,-0.806107
50%,0.073919
75%,0.938462
max,14.444442


## 산타랠리 비교

In [16]:
data21['event'].value_counts()

event
else      2868
before      91
santa       91
after       91
Name: count, dtype: int64

### 일 평균 수익률 비교

In [17]:
data21.groupby('event')[['rtn']].agg(['mean', 'std', 'min', 'max', 'count'])

rtn                                     
            mean       std       min        max count
event                                                
after   0.073416  1.338488 -2.525873   5.624808    91
before  0.119636  1.409404 -4.623483   3.342218    91
else    0.042518  1.620734 -9.116665  14.444442  2868
santa   0.542632  1.526819 -3.989518   3.177795    91

### T-Test & Mann-Whitney U Test

In [18]:
df = data21

rtns_before = df[df['event']=='before']['rtn']
rtns_santa = df[df['event']=='santa']['rtn']
rtns_after = df[df['event']=='after']['rtn']

rtns_before.shape, rtns_santa.shape, rtns_after.shape

((91,), (91,), (91,))

#### H1 : santa > before

In [19]:
res_test = pd.DataFrame(
    columns = ['T-Test', 'mann-Whitney U Test'],
    index = ['Statistic', 'p-value', 'df']
)

In [20]:
res_ttest = ttest_ind(
    rtns_santa, rtns_before, 
    equal_var=True, alternative='greater'
)
t_statistic = res_ttest.statistic
t_pvalue = res_ttest.pvalue
t_df = int(res_ttest.df)

res_test['T-Test'] = [t_statistic, t_pvalue, t_df]

In [21]:
res_utest = mannwhitneyu(
    rtns_santa, rtns_before,
    alternative='greater'
)
u_statistic = res_utest.statistic
u_pvalue = res_utest.pvalue

res_test['mann-Whitney U Test'] = [u_statistic, u_pvalue, '']

In [22]:
# H1 : santa > before

res_test

,T-Test,mann-Whitney U Test
Statistic,1.941942,4996.0
p-value,0.026853,0.008065
df,180.000000,


#### H1 : santa > after

In [23]:
res_test = pd.DataFrame(
    columns = ['T-Test', 'mann-Whitney U Test'],
    index = ['Statistic', 'p-value', 'df']
)

In [24]:
res_ttest = ttest_ind(
    rtns_santa, rtns_after, 
    equal_var=True, alternative='greater'
)
t_statistic = res_ttest.statistic
t_pvalue = res_ttest.pvalue
t_df = int(res_ttest.df)

res_test['T-Test'] = [t_statistic, t_pvalue, t_df]

In [25]:
res_utest = mannwhitneyu(
    rtns_santa, rtns_after,
    alternative='greater'
)
u_statistic = res_utest.statistic
u_pvalue = res_utest.pvalue

res_test['mann-Whitney U Test'] = [u_statistic, u_pvalue, '']

In [26]:
# H1 : santa > after

res_test

,T-Test,mann-Whitney U Test
Statistic,2.204456,5192.0
p-value,0.014380,0.001551
df,180.000000,


### OLS Regression

In [27]:
res_ols = pd.DataFrame(
    index = [
        name, 'T-statistics', 'p-values'
    ],
    columns = [
        'Constant', 'Santa Rally Days', 'F-Value', 'Significance'
    ]
).fillna('')

In [28]:
_df = df[(df['event']=='santa') | (df['event']=='before')]
_df['event'].value_counts()

event
before    91
santa     91
Name: count, dtype: int64

In [29]:
y = _df['rtn']
x = np.where(_df['event']=='santa', 1, 0)
x = sm.add_constant(x)

model = sm.OLS(y, x)
results = model.fit()

res_ols.loc[name] = results.params.values.tolist() + [results.fvalue, results.f_pvalue]
res_ols.loc['T-statistics'].iloc[:2] = results.tvalues.values.tolist()
res_ols.loc['p-values'].iloc[:2] = results.pvalues.values.tolist()

In [33]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    rtn   R-squared:                       0.021
Model:                            OLS   Adj. R-squared:                  0.015
Method:                 Least Squares   F-statistic:                     3.771
Date:                Fri, 05 Jan 2024   Prob (F-statistic):             0.0537
Time:                        16:00:31   Log-Likelihood:                -327.27
No. Observations:                 182   AIC:                             658.5
Df Residuals:                     180   BIC:                             664.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1196      0.154      0.777      0.438      -0.184       0.424
x1             0.4230      0.218      1.942      0.054      -0.007       0.853
==============================================================================
Omnibus:                       10.697   Durbin-Watson:                   1.974
Prob(Omnibus):                  0.005   Jarque-Bera (JB):               11.247
Skew:                          -0.508   Prob(JB):                      0.00361
Kurtosis:                       3.672   Cond. No.                         2.62
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [30]:
res_ols

,Constant,Santa Rally Days,F-Value,Significance
KOSPI,0.119636,0.422996,3.771138,0.053705
T-statistics,0.776743,1.941942,,
p-values,0.43833,0.053705,,
